In [ ]:
# Run this block before anything else

!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install cython
!pip install cartopy

!apt-get -qq install python-cartopy python3-cartopy
!pip uninstall -y shapely    # cartopy and shapely aren't friends (early 2020)
!pip install shapely --no-binary shapely

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np
import scipy.io.netcdf as sp

import cartopy
import cartopy.crs as ccrs                   # for projections
import cartopy.feature as cf           # for features
import cartopy.io.shapereader as shapereader
from cartopy.util import add_cyclic_point
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.feature.nightshade import Nightshade

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# This script creates a global contour map of the mean RSR values at each location

# Open the dataset file
fileObj = sp.NetCDFFile('/content/drive/Shared drives/Research Project Data/CERES_EBAF-TOA_Ed4.1_Subset_200003-202111.nc', 'r')

# Make variables to store all the RSR, latitude, and longitude values
RSR = fileObj.variables['toa_sw_all_mon'][:]
lat = fileObj.variables['lat'][:]
lon = fileObj.variables['lon'][:]

# Calculate how many full years of data is in the dataset
numYears = RSR.shape[0] // 12

# Take a subset of the data in which the extra months at the end that are not part of a full year are excluded
fullYearRSR = RSR[0:numYears * 12, :, :]

# Average the data at each location across all months
means = fullYearRSR.mean(axis=0)

# Fix problem with 0 longitude missing data
means, lon = add_cyclic_point(means, coord=lon)

# Set up a global map using the Plate Carree projection and add coastlines
axes = plt.axes(projection=ccrs.PlateCarree())
axes.add_feature(cf.COASTLINE, linewidth=0.5)

# Overlay the contours onto the map based on the mean RSR values at each location. Include a color bar and add a title
plt.contourf(lon[:], lat[:], means[:,:], levels=12, cmap='bone')
plt.colorbar(orientation='horizontal')
plt.title('Mean RSR (W/m^2), Mar 2000 - Feb 2021')

# Save a PNG image of the map
plt.savefig('/content/drive/My Drive/Research Project for CSATS/RSR_Map.png', bbox_inches='tight', dpi=200)

# projection options: https://scitools.org.uk/cartopy/docs/latest/crs/projections.html
# cmap options: https://matplotlib.org/stable/tutorials/colors/colormaps.html